<a href="https://colab.research.google.com/github/Kuroshio2023/Gaussian-Distance/blob/main/Gaussian_Distance_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl

In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

GPU Available: True
GPU Name: Tesla T4


In [ ]:
import torch
import pandas as pd
import torch.nn.functional as F
import math
from tqdm import tqdm


In [ ]:
from google.colab import files
file_path = "/content/selected_points.xlsx"
df = pd.read_excel(file_path)

# 3️⃣ Extract numeric data (S2 dataset)
data = df[["X_11", "X_12", "X_13", "X_21", "X_22", "X_23"]].to_numpy()
data = torch.tensor(data, dtype=torch.float32, device="cuda")

In [ ]:
# Set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define expectation function in PyTorch
def expectation(mu_1, sigma_1, mu_2, sigma_2):
    pi = torch.tensor(torch.pi)

    A_1 = torch.sqrt(pi)*(2*sigma_1**8+8*sigma_1**6*sigma_2**2+12*sigma_1**4*sigma_2**4+8*sigma_1**2*sigma_2**6+2*sigma_2**8)*sigma_1**2*sigma_2**2
    A_2 = (0.5*sigma_1**2+0.5*sigma_2**2)**1.5

    B_1 = 2 * torch.sqrt(pi)*(0.5*sigma_1**8+2*sigma_1**6*sigma_2**2+3*sigma_1**4*sigma_2**4+2*sigma_1**2*sigma_2**6+0.5*sigma_2**8)
    B_2 = (0.5*sigma_1**2+0.5*sigma_2**2)**0.5

    C = 4 * (sigma_1**8+4*sigma_1**6*sigma_2**2+6*sigma_1**4*sigma_2**4+4*sigma_1**2*sigma_2**6+sigma_2**8)

    Ef_1X= 1/torch.sqrt(2*pi*(sigma_1**2+sigma_2**2))
    Ef_2X= Ef_1X * 0.5
    Ef_3X= (A_1/A_2 + B_1/B_2)/(2*pi*C)

    #Ef_1X= 1/torch.sqrt(2*pi*(sigma_1**2+sigma_2**2))



    return [Ef_1X, Ef_2X, Ef_3X]


# Define the Gaussian Metric Function in PyTorch
def gaussian_metric(u, v):
    """
    Computes Gaussian-based distance between two points u and v.
    """
    n = u.shape[0] // 2  # Half the dimension

    # Splitting input vectors
    u_1, u_2, v_1, v_2 = u[:n], u[n:], v[:n], v[n:]

    # Compute lengths
    l_u = torch.norm(u_1 - u_2)
    l_v = torch.norm(v_1 - v_2)

    # Compute sums for metric formula
    a_1, a_2, a_3 = torch.sum(u_1 ** 2), torch.sum(u_2 ** 2), 2 * torch.sum(u_1 * u_2)
    b_1, b_2, b_3 = torch.sum(v_1 ** 2), torch.sum(v_2 ** 2), 2 * torch.sum(v_1 * v_2)
    c_1 = torch.sum(u_1 * v_1)
    c_2 = torch.sum(u_1 * v_2 + u_2 * v_1)
    c_3 = torch.sum(u_2 * v_2)

    # Expectation values
    p_1, p_2, p_3 = expectation(0.5, l_u / 2, 0.5, l_u / 2)
    q_1, q_2, q_3 = expectation(0.5, l_v / 2, 0.5, l_v / 2)
    r_1, r_2, r_3 = expectation(0.5, l_u / 2, 0.5, l_v / 2)

    # Compute A, B, C
    A = a_1 * p_3 + a_2 * (p_1 + p_3 - 2 * p_2) + a_3 * (p_2 - p_3)
    B = b_1 * q_3 + b_2 * (q_1 + q_3 - 2 * q_2) + b_3 * (q_2 - q_3)
    C = c_1 * r_3 + c_2 * (r_2 - r_3) + c_3 * (r_1 + r_3 - 2 * r_2)
    if A + B - 2 * C>0:

      return torch.sqrt(A + B - 2 * C)
    return A + B - 2 * C

# Generate random data (5000 points in 20 dimensions, split into two halves)
num_points = 600
dim = 6  # Double of original n (for u_1 and u_2)

# Compute pairwise distance matrix using the custom metric
distance_matrix = torch.zeros((num_points, num_points), device=device)

for i in tqdm(range(num_points), desc="Computing distances"):
    for j in range(i, num_points):  # Compute only upper triangle to save time
        distance = gaussian_metric(data[i], data[j])
        if i == j:
            continue
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance  # Symmetric matrix

# Move to CPU and convert to NumPy for saving
distance_matrix_cpu = distance_matrix.cpu().numpy()

# Save as Excel file
df = pd.DataFrame(distance_matrix_cpu)
df.to_excel("distance_matrix.xlsx", index=False, header=False)

print("Custom distance matrix saved as 'distance_matrix.xlsx'.")

Computing distances: 100%|██████████| 600/600 [11:07<00:00,  1.11s/it]


Custom distance matrix saved as 'distance_matrix.xlsx'.


In [ ]:
from google.colab import files
files.download("distance_matrix.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>